# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
# load data from database
def load_data(loc='TrialDatabaseName.db',table='messagelabel'):
    engine = create_engine(f'sqlite:///{loc}')
    df = pd.read_sql_table(table_name=table, con=engine)
    X = df['message']
    cols=list(df.columns)
    for item in ['id','message','original','genre'] :
        cols.remove(item)
    Y = df[cols]
    return X, Y

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return tokens

In [6]:
X,Y = load_data()

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [81]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,make_scorer
from sklearn.metrics import classification_report

In [79]:
def scoring_method(y_pred,y_test):
    return (y_test==y_pred).mean().min()
    
def build_pipeline():
    pipeline = Pipeline([
        ('count',CountVectorizer(tokenizer=tokenize)),
        ('tfid',TfidfTransformer()),
        ('clf',RandomForestClassifier()),
    ])
    
    parameters= {
        'clf__n_estimators': [50, 100] #,200,300,400,600],
        'clf__min_samples_split': [2, 3, 4]
    }
    
    cv=GridSearchCV(pipeline,parameters,scoring=make_scorer(scoring_method))
    return cv

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:


def train_pipeline():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results( y_test, y_pred, cv=model)
    return y_pred,y_test, model

y_pred, y_test, model = train_pipeline()


In [ ]:
def display_results( y_test, y_pred, cv=None):
    print("\nBest Parameters:", cv.best_params_
    accuracy = (y_pred == y_test).mean()
    print('Accuracy Results in each Category \n \n',accuracy)
    print('------------------------')
    for a,b in zip(y_test, y_pred.transpose()):
        confusion_mat = confusion_matrix(y_test[a].values, b)
        print("categories:"+str(a)+'\n',confusion_mat)
        print('\n')

In [ ]:
display_results(y_test,y_pred,cv=model)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [68]:
def display_classification( y_test, y_pred):
    for a,b in zip(y_test, y_pred.transpose()):
        print("Classification_report: "+str(a)+'\n',
             classification_report(y_test[a].values,b))
        print('\n')

display_classification(y_test,y_pred)  

Classification_report: related
              precision    recall  f1-score   support

          0       0.61      0.51      0.56      1504
          1       0.86      0.90      0.88      5010
          2       0.22      0.35      0.27        40

avg / total       0.80      0.81      0.80      6554



Classification_report: request
              precision    recall  f1-score   support

          0       0.90      0.98      0.94      5424
          1       0.82      0.47      0.59      1130

avg / total       0.88      0.89      0.88      6554



Classification_report: offer
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554



Classification_report: aid_related
              precision    recall  f1-score   support

          0       0.70      0.91      0.79      3834
          1       0.79      0.45      0.57      2720

avg /

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
def export_model(model,name='trained.model'):
    fp=open(name,'wb')
    fp.write(pickle.dumps(model))
    fp.close()
    

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [1]:
%%writefile train.py
# import packages
# import for data loading and text handling
import sys
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
#nltk library imports
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#sklearn libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,make_scorer
from sklearn.metrics import classification_report
#sklearn ML libraries
from sklearn.ensemble import RandomForestClassifier

def load_data(loc,table='categorizedMessages'):
    # read in file
    # clean data
    # load to database
    engine = create_engine(f'sqlite:///{loc}')
    conn=engine.connect()
    df = pd.read_sql_table(table_name=table, con=conn)
    # define features and label arrays
    X = df['message']
    cols=list(df.columns)
    for item in ['id','message','original','genre'] :
        cols.remove(item)
    Y = df[cols]
    return X, Y

def tokenize(text):
    '''function to create word tokens for text '''
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return tokens

def scoring_method(y_pred,y_test):
    '''function to score results by GridSearch'''
    return (y_test==y_pred).mean().min()


def build_model():
    # text processing and model pipeline
    pipeline = Pipeline([
        ('count',CountVectorizer(tokenizer=tokenize)),
        ('tfid',TfidfTransformer()),
        ('clf',RandomForestClassifier(n_estimators=200, min_samples_split=3)),
    ])
    #  parameters for GridSearchCV
    '''parameters= {
        'clf__n_estimators': [50, 100 ,], # 200,300,400,600],
        'clf__min_samples_split': [2, 3,]# 4],
    }'''
    # gridsearch object and return as final model pipeline
    '''pipeline=GridSearchCV(pipeline,
                    parameters,
                    scoring=make_scorer(scoring_method))'''

    return pipeline

def display_results( y_test, y_pred, cv=None):
    '''helper function to plot resultsc'''
    #print("\nBest Parameters:", cv.best_params_)
    accuracy = (y_pred == y_test).mean()
    print('Accuracy Results in each Category \n \n',accuracy)
    print('------------------------')
    for a,b in zip(y_test, y_pred.transpose()):
        confusion_mat = confusion_matrix(y_test[a].values, b)
        print("categories:"+str(a)+'\n',confusion_mat)
        print('\n')


def train(X, y, model):
    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    # fit model
    model.fit(X_train, y_train)
    # output model test results
    y_pred = model.predict(X_test)
    display_results( y_test, y_pred, cv=None)
    return model


def export_model(model,filename='model.pkl'):
    fp=open(filename,'wb')
    fp.write(pickle.dumps(model))
    fp.close()
    return True


def run_pipeline(data_file):
    X, y = load_data(data_file)  # run ETL pipeline
    model = build_model()  # build model pipeline
    model = train(X, y, model)  # train model pipeline
    export_model(model)  # save model
    return True


if __name__ == '__main__':
    data_file = sys.argv[1]  # get filename of dataset
    run_pipeline(data_file)  # run data pipeline

Overwriting train.py


In [2]:
from train import load_data, train, run_pipeline, export_model, build_model, scoring_method

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
run_pipeline('LabelledMessages.db')

Accuracy Results in each Category 
 
 related                   0.822551
request                   0.896094
offer                     0.995117
aid_related               0.763808
medical_help              0.921269
medical_products          0.947208
search_and_rescue         0.972994
security                  0.984437
military                  0.968721
child_alone               1.000000
water                     0.950565
food                      0.927067
shelter                   0.926152
clothing                  0.986573
money                     0.976961
missing_people            0.989777
refugees                  0.969332
death                     0.960635
other_aid                 0.868782
infrastructure_related    0.936070
transport                 0.958499
buildings                 0.951785
electricity               0.981691
tools                     0.992676
hospitals                 0.989625
shops                     0.994660
aid_centers               0.985963
other_infrastruct

In [ ]:
!zip -./.